In [1]:
from itertools import product

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from download_data import get_price, get_mpd, get_fx
from data_analysis.explore_data import (
    calc_rolling_stat,
    merge_prob_stat,
    rename_stat_df,
    eval_large_change_prob,
    plot_change_scatter,
    prep_regression_stat,
    compare_regressions,
    compare_regression_eval
)

In [2]:
mpd = get_mpd()
start_date, end_date = mpd["idt"].min(), mpd["idt"].max()

equity = get_price(tickers=["BAC", "C", "IYR", "SPY"], 
                  start_date=start_date, 
                  end_date=end_date, 
                  use_cache=False)

fx = get_fx(tickers=["GBP", "EUR", "JPY"],
            start_date=start_date,
            end_date=end_date,
            use_cache=False)

In [6]:
fx

code,EUR,GBP,JPY
date,,,
2006-01-12,0.829958,0.567039,113.974697
2006-01-13,0.825866,0.564469,114.457586
2006-01-14,0.827567,0.564547,114.464902
2006-01-15,0.827567,0.564547,114.464902
2006-01-16,0.825142,0.565478,114.801784
...,...,...,...
2024-01-06,0.912850,0.785855,144.635017
2024-01-07,0.912850,0.785855,144.635017
2024-01-08,0.914231,0.787117,144.344000


In [3]:
stat = calc_rolling_stat(pd.concat([fx, equity], axis=1), month=6)
market_list = ["bac", "citi", "iyr", "sp6m", "sp12m", "euro", "pound", "yen"]
ticker_list = ["BAC", "C", "IYR", "SPY", "SPY", "GBP", "EUR", "JPY"]
stat_rename = rename_stat_df(stat, dict(zip(market_list, ticker_list)))

In [4]:
df = merge_prob_stat(stat_rename, mpd)

In [5]:
def calc_hit_rate(df: pd.DataFrame, idx: list[str]=market_list):
    p_col = ["p10", "p50", "p90"]
    for c in p_col:
        df.loc[:, f"{c}_hit"] = df[c] > df["ret"]
    return df.filter(regex="_hit$").groupby("market").mean().reindex(market_list)

calc_hit_rate(df)

,p10_hit,p50_hit,p90_hit
market,,,
bac,0.122924,0.413621,0.790698
citi,0.098007,0.471761,0.837209
iyr,0.058076,0.372051,0.838475
sp6m,0.022288,0.380386,0.979198
sp12m,0.008915,0.433878,1.000000
euro,0.096831,0.496479,0.786972
pound,0.056338,0.447183,0.839789
yen,0.082746,0.422535,0.801056
